In [1]:
import pandas as pd
import math

In [2]:
input_data = {
    'sieve_size': [4, 2, 1, 0.5, 0.25, 0.125, 0.025],
    'soil_retained_%': [0.5, 2, 40, 50, 75, 25, 7.5]
}

In [3]:
df = pd.DataFrame(input_data)

In [4]:
df

,sieve_size,soil_retained_%
0,4.000,0.5
1,2.000,2.0
2,1.000,40.0
3,0.500,50.0
4,0.250,75.0
5,0.125,25.0
6,0.025,7.5


In [5]:
def log(value):
    return -math.log(value, 2)

In [6]:
df['phi_scale'] = df['sieve_size'].apply(log).round(decimals=2)

In [7]:
# re-order the columns and handle negative zeros
df = df.reindex(columns=['sieve_size', 'phi_scale', 'soil_retained_%'])
df[df == -0] = 0

In [8]:
df

,sieve_size,phi_scale,soil_retained_%
0,4.000,-2.00,0.5
1,2.000,-1.00,2.0
2,1.000,0.00,40.0
3,0.500,1.00,50.0
4,0.250,2.00,75.0
5,0.125,3.00,25.0
6,0.025,5.32,7.5


In [9]:
total_soil_retained = df['soil_retained_%'].sum()
total_soil_retained

200.0

In [10]:
df['percent_retained_%'] = (df['soil_retained_%'] / total_soil_retained) * 100
df['cumulative_retained_%'] = df['percent_retained_%'].cumsum()
df['percent_passing_%'] = 100 - df['cumulative_retained_%']

In [11]:
df

,sieve_size,phi_scale,soil_retained_%,percent_retained_%,cumulative_retained_%,percent_passing_%
0,4.000,-2.00,0.5,0.25,0.25,99.75
1,2.000,-1.00,2.0,1.00,1.25,98.75
2,1.000,0.00,40.0,20.00,21.25,78.75
3,0.500,1.00,50.0,25.00,46.25,53.75
4,0.250,2.00,75.0,37.50,83.75,16.25
5,0.125,3.00,25.0,12.50,96.25,3.75
6,0.025,5.32,7.5,3.75,100.00,0.00
